In [3]:
import ipywidgets as widgets

widgets.Text(
    value='Test',
    placeholder='Enter your event code',
    description='Event Code:',
    disabled=False
)

Text(value='Test', description='Event Code:', placeholder='Enter your event code')

In [28]:
import pandas as pd
df = pd.read_csv('test.csv').head()

import tempfile
with tempfile.NamedTemporaryFile() as temp:
    df.to_csv(temp.name+'.csv')

In [2]:
from Google import Create_Service

CLIENT_SECRET_FILE = 'client_secret.json'
API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

client_secret.json-drive-v3-(['https://www.googleapis.com/auth/drive'],)
['https://www.googleapis.com/auth/drive']
drive service created successfully


In [36]:
from datetime import datetime
timestamp_str = datetime.now().strftime('%Y%m%d_%H%m')

from googleapiclient.http import MediaFileUpload

folder_id = '1UogxFBwmbd6PXqQ40Wtp8HlAFxAspTFw'
request = service.files().create(
    body={
        'name':f'test_{timestamp_str}.csv',
        'parents':[folder_id]
    },
    media_body=MediaFileUpload(
        temp.name+'.csv',
        mimetype='application/octet-stream',
        resumable=True
    )
)

response = None
while response is None:
    status, response = request.next_chunk()
    if status:
        print("Uploaded %d%%." % int(status.progress() * 100))
print("Upload Complete!")

Upload Complete!
